In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display # Allows the use of display() for DataFrames
import seaborn as sb
%matplotlib inline
# from keras.applications.densenet import DenseNet121

import keras
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Input, Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, Add, Concatenate
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
import timeit
import os
from datetime import datetime
import tensorflow as tf
from math import ceil
from main_v2 import (generate_dataframe_from_csv_horizontal, generate_dataframe_from_csv_vertical, 
                         get_model_inputs, create_multi_generator, build_model, ConvBlock)
# https://github.com/keras-team/keras/issues/4161#issuecomment-366031228
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

# import wandb
# from wandb.keras import WandbCallback
from train import TrainingRunner

Using TensorFlow backend.


In [2]:
# df = generate_dataframe_from_csv_vertical("train.csv", inputs=1)
# df = generate_dataframe_from_csv_horizontal("train.csv")
df = pd.read_csv("./train_1.csv",  dtype={'sirna': object})
val_split = 0.25
epochs = 20
batch_size = 32
train_size = int(len(df)*(1-val_split))
valid_size = int(len(df)*(val_split))
steps = ceil(train_size/batch_size)
steps_valid = ceil(valid_size/batch_size)
print(train_size, valid_size, steps, steps_valid)
# display(df)

54772 18257 1712 571


In [3]:
# runner = TrainingRunner(filename="train_root.csv", use_multi=True, epochs=epochs, batch_size=batch_size, val_split=val_split, use_wandb=False)
runner = TrainingRunner(batch_size=8)

Found 58424 validated image filenames belonging to 1108 classes.
Found 14606 validated image filenames belonging to 1108 classes.
Instructions for updating:
Colocations handled automatically by placer.


C:\ProgramData\Anaconda3\envs\ml\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 6s 0us/step
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              100353000 
_________________________________________________________________
bn_fc_1 (BatchNormalization) (None, 1000)              4000      
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_2 (Dense)             

In [4]:
# runner.summary()

In [ ]:
runner.run()

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
7303/7303 [==============================] - 772s 106ms/step - loss: 7.2250 - acc: 8.2158e-04 - val_loss: 14.9397 - val_acc: 8.2158e-04

Epoch 00001: val_loss improved from inf to 14.93974, saving model to saved_models/weights.best.from_scratch.hdf5
Epoch 2/20
7303/7303 [==============================] - 717s 98ms/step - loss: 7.0213 - acc: 0.0012 - val_loss: 12.9612 - val_acc: 8.9005e-04

Epoch 00002: val_loss improved from 14.93974 to 12.96116, saving model to saved_models/weights.best.from_scratch.hdf5
Epoch 3/20
7303/7303 [==============================] - 706s 97ms/step - loss: 6.9984 - acc: 0.0016 - val_loss: 13.0310 - val_acc: 8.9005e-04

Epoch 00003: val_loss did not improve from 12.96116
Epoch 4/20
7303/7303 [==============================] - 707s 97ms/step - loss: 6.9677 - acc: 0.0017 - val_loss: 12.2175 - val_acc: 9.5851e-04

Epoch 00004: val_loss improved from 12.96116 to 12.21754, saving model to saved_models/weigh

In [3]:
# wandb.init(project="testing-ml")

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, 
                                   verbose=1, mode='auto', min_delta=0.0001)

early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=15)

csv_logger = CSVLogger(filename='./training_log.csv',
                       separator=',',
                       append=True)

logdir = os.path.join("logs", datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)


(images, trainY) = get_model_inputs(df)

train_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=val_split)

# train_generator = create_multi_generator(df, train_datagen, "training") 
# valid_generator = create_multi_generator(df, train_datagen, "validation") 
train_generator = train_datagen.flow_from_dataframe(
        df,
        directory="./",
        x_col="img_path",
        y_col="sirna",
        target_size=(224, 224),
        batch_size=batch_size,
        subset="training",
        class_mode='categorical')

valid_generator = train_datagen.flow_from_dataframe(
        df,
        directory="./",
        x_col="img_path",
        y_col="sirna",
        target_size=(224, 224),
        batch_size=batch_size,
        subset="validation",
        class_mode='categorical')

Found 54773 validated image filenames belonging to 1108 classes.
Found 18257 validated image filenames belonging to 1108 classes.


In [4]:
model = build_model()
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      4864      
_________________________________________________________________
batch_normalization_1 (Batch (None, 224, 224, 64)      256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
batch_normalization_2

In [ ]:
#https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)
# WORKERS = 2

model.fit_generator(train_generator, 
                    steps_per_epoch=steps,
                    validation_data=valid_generator,
                    validation_steps=steps_valid,
#                     workers=WORKERS, 
#                     use_multiprocessing=True,
                    epochs=epochs, callbacks=[checkpointer,reduceLROnPlat, early, csv_logger, tensorboard_callback], verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
1712/1712 [==============================] - 678s 396ms/step - loss: 7.5768 - acc: 0.0012 - val_loss: 16.4520 - val_acc: 8.2160e-04

Epoch 00001: val_loss improved from inf to 16.45205, saving model to saved_models/weights.best.from_scratch.hdf5
Epoch 2/20
  41/1712 [..............................] - ETA: 9:26 - loss: 7.3360 - acc: 0.0023

In [ ]:
#!wandb run python train.py

In [ ]:
#tensorboard --logdir=logs/